In [2]:
import os
from glob import glob
import numpy as np
import pandas
from difflib import get_close_matches

In [4]:
probes = pandas.read_csv('/home/users/jvogel/Science/Allen_Human_Brain_Atlas/normalized_microarray_donor10021/Probes.csv')

#### Get other studies

In [35]:
studies = pandas.ExcelFile('/Users/jakevogel/Dropbox/Work/Projects/AHBA_Hippocampus_AP/Other_papers.xlsx'
                          ).parse('Sheet1')
studies.head()

,Oreilly_D,Oreilly_D_entrez,Oreilly_V,Oreilly_V_entrez,Cembrowski2017_V,Cembrowski2017_D,Thompson,Christensen_v,Christensen_d,Dong
0,Cyp1b1,25426.0,Caln1,363909.0,Cntnap5c,Cdh7,Sema5b,Sstr1,Asam,Col5a1
1,Cyp26b1,312495.0,Crtac1,171438.0,Ntng2,Emb,Sema5a,Rab3b,Loc498835,Dcn
2,Ntf3,81737.0,Dcn,29139.0,Pcdh20,Fn1,Robo1,Odz4,Nt5,Matn2
3,NTS,299757.0,Gda,83585.0,Robo1,Nts,Ephb1,Stk23,Plscr1,Fbln2
4,Olfml2b,304960.0,Cadm1,363058.0,Sema3a,Rxfp1,Epha7,Fxyd6,Ppl,Lct


#### Prepare Leonardos

In [4]:
leo = pandas.ExcelFile('/Users/jakevogel/Dropbox/Work/Projects/AHBA_Hippocampus_AP/Leonardo_2006_Table.xls'
                      ).parse('Sheet1')

In [7]:
leo_d = leo[leo['V/D Fold change']<-2.529]['Name'].values
leo_v = leo[leo['V/D Fold change']>3.13]['Name'].values

In [48]:
for i,gene in enumerate(leo_d):
    matches = get_close_matches(gene, probes.gene_name.unique())
    #print(gene,'\n',matches,'\n\n')
    if len(matches)>0:
        hit = probes[probes['gene_name']==matches[0]
                    ]['gene_symbol'].values[0]
        studies.loc[i,'Leonardo_2006_dorsal'] = hit


In [47]:
for i,gene in enumerate(leo_v):
    matches = get_close_matches(gene, probes.gene_name.unique())
    query = True
    print(gene,'\n',matches,'\n\n')
    if len(matches)>0:
        while query:
            ans = input('Keep it? ')
            if ans == 'y':
                hit = probes[probes['gene_name']==matches[0]
                            ]['gene_symbol'].values[0]
                studies.loc[i,'Leonardo_2006_ventral'] = hit
            query = False
            


RIKEN cDNA 2810454P21 gene 
 [] 


regulated endocrine-specific protein 18 
 ['regulated endocrine-specific protein 18 homolog (rat)', 'lymphocyte-specific protein 1', 'NOTCH-regulated ankyrin repeat protein'] 


Keep it? y
coatomer protein complex, subunit gamma 2, antisense 2 
 ['coatomer protein complex, subunit gamma 2', 'coatomer protein complex, subunit gamma 1', 'coatomer protein complex, subunit zeta 2'] 


Keep it? y
tissue inhibitor of metalloproteinase 2 
 ['X-linked inhibitor of apoptosis'] 


Keep it? n
coatomer protein complex, subunit gamma 2, antisense 2 
 ['coatomer protein complex, subunit gamma 2', 'coatomer protein complex, subunit gamma 1', 'coatomer protein complex, subunit zeta 2'] 


Keep it? y
RIKEN cDNA 2210017A09 gene 
 [] 


slit homolog 2 (Drosophila) 
 ['slit homolog 2 (Drosophila)', 'slit homolog 3 (Drosophila)', 'slit homolog 1 (Drosophila)'] 


Keep it? y
expressed sequence AI527257 
 [] 


tissue inhibitor of metalloproteinase 2 
 ['X-linked inhibitor 

In [19]:
allvals = sorted(abs(leo['V/D Fold change'].values)[1:])
onep = len(allvals) - int(len(allvals)*0.01)
vmax = allvals[onep]
leo_thr = leo[abs(leo['V/D Fold change']) > vmax]
leo_d_1p = []
leo_v_1p = []
for i,row in leo_thr.iterrows():
    if row['V/D Fold change'] > 0:
        leo_v_1p.append(leo_thr.loc[i,'Name'])
    else:
        leo_d_1p.append(leo_thr.loc[i,'Name'])

In [73]:
studies = studies.reindex(index=range(300))

In [82]:
#busts = []
for i,gene in enumerate(leo_v_1p):
    if gene in busts:
        continue
    if pandas.notnull(studies.loc[i,'Leonardo_2006_ventral_top1p']):
        continue
    if not pandas.notnull(gene):
        continue
    matches = get_close_matches(gene, probes.gene_name.unique())
    query = True
    print(gene,'\n',matches,'\n\n')
    if len(matches)>0:
        while query:
            ans = input('Keep it? ')
            if ans == 'y':
                hit = probes[probes['gene_name']==matches[0]
                            ]['gene_symbol'].values[0]
                studies.loc[i,'Leonardo_2006_ventral_top1p'] = hit
            elif ans.isdigit():
                hit = probes[probes['gene_name']==matches[int(ans)]
                            ]['gene_symbol'].values[0]
                studies.loc[i,'Leonardo_2006_ventral_top1p'] = hit
            elif len(ans) > 2:
                hit = probes[probes['gene_name']==ans
                            ]['gene_symbol'].values[0]
                studies.loc[i,'Leonardo_2006_ventral_top1p'] = hit
            else:
                busts.append(gene)
            query = False
            

RIKEN cDNA B930011D01 gene 
 [] 


RIKEN cDNA B430320C24 gene 
 [] 


differential display and activated by p53 
 [] 


RIKEN cDNA 2410022L05 gene 
 [] 


RIKEN cDNA 1810037C22 gene 
 [] 


RIKEN cDNA 4930434H03 gene 
 [] 


expressed sequence AI315208 
 [] 


RIKEN cDNA 1300010A20 gene 
 [] 


Mus musculus, clone IMAGE:4161424, mRNA 
 [] 


RIKEN cDNA 0710001E13 gene 
 [] 


RIKEN cDNA 1300002C13 gene 
 [] 


RIKEN cDNA 4933440J18 gene 
 [] 


RIKEN cDNA A330103B05 gene 
 [] 


RIKEN cDNA 1300002C13 gene 
 [] 


ESTs, Highly similar to JN0609 nitric-oxide synthase (EC 1.14.13.39), neuronal - mouse [M.musculus] 
 [] 


expressed sequence AA536730 
 [] 


RIKEN cDNA 2310047E01 gene 
 [] 


RIKEN cDNA 9030408N13 gene 
 [] 


ESTs 
 [] 


RIKEN cDNA A330103B05 gene 
 [] 


RIKEN cDNA 2900075G08 gene 
 [] 


RIKEN cDNA A330103B05 gene 
 [] 


RIKEN cDNA 2410012A13 gene 
 [] 


5-hydroxytryptamine (serotonin) receptor 2C 
 ['5-hydroxytryptamine (serotonin) receptor 3C, ionotropic', '5-hydro

In [114]:
busts = []
for i,gene in enumerate(leo_d_1p):
    if gene in busts:
        continue
    if pandas.notnull(studies.loc[i,'Leonardo_2006_dorsal_top1p']):
        continue
    if not pandas.notnull(gene):
        continue
    matches = get_close_matches(gene, probes.gene_name.unique())
    query = True
    print(gene,'\n',matches,'\n\n')
    if len(matches)>0:
        while query:
            ans = input('Keep it? ')
            if ans == 'y':
                hit = probes[probes['gene_name']==matches[0]
                            ]['gene_symbol'].values[0]
                studies.loc[i,'Leonardo_2006_dorsal_top1p'] = hit
            elif ans.isdigit():
                hit = probes[probes['gene_name']==matches[int(ans)]
                            ]['gene_symbol'].values[0]
                studies.loc[i,'Leonardo_2006_dorsal_top1p'] = hit
            elif len(ans) > 2:
                hit = probes[probes['gene_name']==ans
                            ]['gene_symbol'].values[0]
                studies.loc[i,'Leonardo_2006_dorsal_top1p'] = hit
            else:
                busts.append(gene)
            query = False
            

ESTs, Weakly similar to  klotho [Mus musculus] [M.musculus] 
 [] 


RIKEN cDNA 4930526B11 gene 
 [] 


RIKEN cDNA 1190002N15 gene 
 [] 


RIKEN cDNA 2810011M08 gene 
 [] 


RIKEN cDNA 4930526B11 gene 
 [] 


Mus musculus, Similar to RAS protein-specific guanine nucleotide-releasing factor 1, clone MGC:36047 IMAGE:5367388, mRNA, complete cds 
 [] 


RIKEN cDNA 1190002N15 gene 
 [] 


RIKEN cDNA 1200009K10 gene 
 [] 


RIKEN cDNA 1200017E04 gene 
 [] 


brain and heart sodium channel beta 3 subunit 
 [] 


RIKEN cDNA 4833414B02 gene 
 [] 


RIKEN cDNA 2310043N10 gene 
 [] 


RIKEN cDNA 4933402K10 gene 
 [] 


RIKEN cDNA 6330415F13 gene 
 [] 


RIKEN cDNA 3110003A17 gene 
 [] 


RIKEN cDNA 6330415F13 gene 
 [] 


RIKEN cDNA 2700050C12 gene 
 [] 


RIKEN cDNA 6330527O06 gene 
 [] 


solute carrier family 17 (vesicular glutamate transporter), member 6 
 ['solute carrier family 1 (glutamate transporter), member 7', 'solute carrier family 32 (GABA vesicular transporter), member 1', 'solute ca

In [116]:
studies.to_csv('/Users/jakevogel/Science/AHBA/Other_Studies_Genes.csv')

#### Prepare Lees

In [124]:
lee_allv = pandas.ExcelFile('/Users/jakevogel/Dropbox/Work/Projects/AHBA_Hippocampus_AP/Lee_2017_TableS2.xlsx'
                           ).parse('Ventrally-enriched genes')
lee_allv.columns = lee_allv.loc[1,:]
lee_allv.drop([0,1],inplace=True)
lee_alld = pandas.ExcelFile('/Users/jakevogel/Dropbox/Work/Projects/AHBA_Hippocampus_AP/Lee_2017_TableS2.xlsx'
                           ).parse('Dorsally-enriched genes') 
lee_alld.columns = lee_alld.loc[1,:]
lee_alld.drop([0,1],inplace=True)

leev = lee_allv.gene_symbol.values
studies.loc[:,'lee2017_ventral'] = leev.tolist() + ([np.nan]*(len(studies)-len(leev)))
leed = lee_alld.gene_symbol.values
studies.loc[:,'lee2017_dorsal'] = leed.tolist() + ([np.nan]*(len(studies)-len(leed)))

In [127]:
studies = studies.reindex(index=range(1000))

In [128]:
leeall = pandas.ExcelFile('/Users/jakevogel/Dropbox/Work/Projects/AHBA_Hippocampus_AP/Lee_2017_TableS1.xlsx')
for sheet in leeall.sheet_names:
    df = leeall.parse(sheet)
    df.columns = df.loc[1,:]
    df.drop([0,1],inplace=True)
    leev = df[df['%s_log2FC'%sheet]>0].gene_symbol.values
    studies.loc[:,'lee2017_ventral_%s'%sheet] = leev.tolist() + ([np.nan]*(len(studies)-len(leev)))
    leed = df[df['%s_log2FC'%sheet]<0].gene_symbol.values
    studies.loc[:,'lee2017_dorsal_%s'%sheet] = leev.tolist() + ([np.nan]*(len(studies)-len(leev)))

In [130]:
studies.to_csv('/Users/jakevogel/Science/AHBA/Other_Studies_Genes.csv')

In [131]:
studies.to_csv('/Users/jakevogel/Dropbox/Work/Projects/AHBA_Hippocampus_AP/MANUSCRIPT/Natures/NatureComms/Revision/Other_Studies_Genes.csv')

#### Cembrowskis

In [137]:
cem = pandas.ExcelFile('/Users/jakevogel/Dropbox/Work/Projects/AHBA_Hippocampus_AP/Cembrowski_2013_Table.xls'
                     ).parse('tableS2R1.txt')
cemv = cem[cem.enrichedRegion=='ventral']['geneShortName'].values
studies.loc[:,'Cembrowski2013_ventral'] = cemv.tolist() + ([np.nan]*(len(studies)-len(cemv)))
cemd = cem[cem.enrichedRegion=='dorsal']['geneShortName'].values
studies.loc[:,'Cembrowski2013_dorsal'] = cemd.tolist() + ([np.nan]*(len(studies)-len(cemd)))

In [138]:
studies.to_csv('/Users/jakevogel/Dropbox/Work/Projects/AHBA_Hippocampus_AP/MANUSCRIPT/Natures/NatureComms/Revision/Other_Studies_Genes.csv')

In [141]:
studies.drop([x for x in studies.columns if 'entrez' in x],axis=1,inplace=True)

### Compare to my study

In [149]:
gene

'GAL'

In [154]:
set1

array(['GAL', 'A_24_P102119', 'ONECUT2', 'PDLIM5', 'COL5A2', 'KDELR3',
       'GREM2', 'HPSE2', 'DDC', 'CTXN3', 'FAM43B', 'A_24_P62668', 'PVALB',
       'CASR', 'WNT10A', 'TTR', 'FSTL4', 'TNNT2', 'A_32_P121537',
       'A_24_P401842', 'DGKI', 'C1orf133', 'RP11-291L15.2', 'NTN1',
       'OSBPL3', 'TPBG', 'BNC2', 'BDKRB1', 'RGMA', 'GRHL2',
       'RP11-561O23.6', 'HHIP', 'SERTAD4', 'A_32_P11262', 'NPNT',
       'SERPINF1', 'KLK7', 'RSPH9', 'LYPD1', 'GABRQ', 'PYDC1', 'SYTL2',
       'RP13-102H20.1', 'SSTR1', 'LXN', 'TMEM215', 'C1QL1', 'PIRT',
       'KCNG1', 'AQP3', 'A_23_P213527', 'RP11-45B20.3', 'C1orf187',
       'GPR39', 'OPRK1', 'CD36', 'MYB', 'GPR26', 'LMO1', 'GPR88', 'VGLL3',
       'NR2F2', 'GPR83', 'GDA', 'NRG1', 'RSPO2', 'A_32_P136776',
       'C20orf103', 'SYTL1', 'LGALS2', 'KIAA1772', 'A_32_P115840',
       'SEMA3D', 'GEFT'], dtype=object)

array(['SERPINF1', 'KLK7', 'RSPH9', 'LYPD1', 'GABRQ', 'PYDC1', 'SYTL2',
       'RP13-102H20.1', 'SSTR1', 'LXN', 'TMEM215', 'C1QL1', 'PIRT',
       'KCNG1', 'AQP3', 'A_23_P213527', 'RP11-45B20.3', 'C1orf187',
       'GPR39', 'OPRK1', 'CD36', 'MYB', 'GPR26', 'LMO1', 'GPR88', 'VGLL3',
       'NR2F2', 'GPR83', 'GDA', 'NRG1', 'RSPO2', 'A_32_P136776',
       'C20orf103', 'SYTL1', 'LGALS2', 'KIAA1772', 'A_32_P115840', 'SEMA3D'], dtype=object)

In [26]:
me = pandas.read_csv('/home/users/jvogel/Science/Allen_Human_Brain_Atlas/MAIN_model_genes_of_importance.csv',
                    index_col=0)
set1a = me[(me.step==1) & (me['ant-post']=='anterior')].gene_symbol.unique()
set1p = me[(me.step==1) & (me['ant-post']=='posterior')].gene_symbol.unique()

set1a = sorted([x for x in set1a if '_' not in x and '-' not in x])
set1p = sorted([x for x in set1p if '_' not in x and '-' not in x])


for i in range(len(set1a)):
    if i < len(set1p):
        print('%s & & %s & \\\\'%(set1a[i],set1p[i]))
    else:
        print('%s & & & \\\\'%(set1a[i]))    

AQP3 & & BDKRB1 & \\
C1QL1 & & BNC2 & \\
C1orf187 & & C1orf133 & \\
C20orf103 & & CASR & \\
CD36 & & COL5A2 & \\
GABRQ & & CTXN3 & \\
GDA & & DDC & \\
GPR26 & & DGKI & \\
GPR39 & & FAM43B & \\
GPR83 & & FSTL4 & \\
GPR88 & & GAL & \\
KCNG1 & & GEFT & \\
KIAA1772 & & GREM2 & \\
KLK7 & & GRHL2 & \\
LGALS2 & & HHIP & \\
LMO1 & & HPSE2 & \\
LXN & & KDELR3 & \\
LYPD1 & & NPNT & \\
MYB & & NTN1 & \\
NR2F2 & & ONECUT2 & \\
NRG1 & & OSBPL3 & \\
OPRK1 & & PDLIM5 & \\
PIRT & & PVALB & \\
PYDC1 & & RGMA & \\
RSPH9 & & SERTAD4 & \\
RSPO2 & & TNNT2 & \\
SEMA3D & & TPBG & \\
SERPINF1 & & TTR & \\
SSTR1 & & WNT10A & \\
SYTL1 & & & \\
SYTL2 & & & \\
TMEM215 & & & \\
VGLL3 & & & \\


In [150]:
me = pandas.read_csv('/Users/jakevogel/Science/AHBA/MAIN_model_genes_of_importance.csv',
                    index_col=0)
set1 = me[me.step==1].gene_symbol.unique()
for gene in set1:
    print('***',gene,'***')
    for col in studies.columns:
        slist = [str(x).upper() for x in studies[col].values if pandas.notnull(x)]
        if gene.upper() in slist:
            print(col)
    print('\n\n')

*** GAL ***



*** A_24_P102119 ***



*** ONECUT2 ***



*** PDLIM5 ***



*** COL5A2 ***
lee2017_ventral_P14_VH_vs_DH
lee2017_dorsal_P14_VH_vs_DH
lee2017_ventral_P45_VH_vs_DH
lee2017_dorsal_P45_VH_vs_DH



*** KDELR3 ***



*** GREM2 ***



*** HPSE2 ***



*** DDC ***



*** CTXN3 ***



*** FAM43B ***



*** A_24_P62668 ***



*** PVALB ***



*** CASR ***



*** WNT10A ***



*** TTR ***
Leonardo_2006
Leonardo_2006_dorsal
Leonardo_2006_ventral
Leonardo_2006_dorsal_top1p
lee2017_ventral_P28_VH_vs_DH
lee2017_dorsal_P28_VH_vs_DH
lee2017_ventral_P45_VH_vs_DH
lee2017_dorsal_P45_VH_vs_DH



*** FSTL4 ***
lee2017_dorsal



*** TNNT2 ***



*** A_32_P121537 ***



*** A_24_P401842 ***



*** DGKI ***
lee2017_dorsal



*** C1orf133 ***



*** RP11-291L15.2 ***



*** NTN1 ***



*** OSBPL3 ***



*** TPBG ***
Christensen_v
lee2017_ventral_P14_VH_vs_DH
lee2017_dorsal_P14_VH_vs_DH
lee2017_ventral_P45_VH_vs_DH
lee2017_dorsal_P45_VH_vs_DH



*** BNC2 ***



*** BDKRB1 ***



*** RGMA ***



**

In [163]:
set2 = me[me.step==2].gene_symbol.unique()
set2 = list(set(set2) - set(set1))
set2_hits = {}
for gene in set2:
    hits = []
    for col in [x for x in studies.columns if '_P' not in x]:
        slist = [str(x).upper() for x in studies[col].values if pandas.notnull(x)]
        if gene.upper() in slist:
            hits.append(col)
    if len(hits)>0:
        ap = me[me.gene_symbol==gene]['ant-post'].values[0]
        print('***',gene,'***','<%s>'%ap)
        print(hits,'\n\n')
        set2_hits.update({gene: hits})

*** NPTX1 *** <anterior>
['Cembrowski2017_V'] 


*** MEGF11 *** <anterior>
['Cembrowski2013_ventral'] 


*** ATP2B4 *** <anterior>
['lee2017_ventral', 'Cembrowski2013_ventral'] 


*** PLXNC1 *** <anterior>
['Leonardo_2006_ventral_top1p', 'Cembrowski2013_ventral'] 


*** MARCH1 *** <anterior>
['lee2017_ventral'] 


*** MET *** <anterior>
['Leonardo_2006_ventral_top1p', 'Cembrowski2013_ventral'] 


*** HTR3A *** <anterior>
['Christensen_v', 'lee2017_ventral'] 


*** GRP *** <anterior>
['Dong', 'lee2017_ventral', 'Cembrowski2013_ventral'] 


*** HR *** <posterior>
['lee2017_dorsal'] 


*** KCNK2 *** <posterior>
['Leonardo_2006_dorsal_top1p'] 


*** HTR2C *** <posterior>
['Dong', 'Leonardo_2006_ventral_top1p', 'lee2017_ventral', 'Cembrowski2013_ventral'] 


*** KCNH5 *** <posterior>
['lee2017_dorsal'] 


*** HDC *** <posterior>
['Christensen_d', 'lee2017_dorsal'] 


*** USP2 *** <posterior>
['lee2017_dorsal'] 


*** CARTPT *** <posterior>
['lee2017_dorsal'] 


*** IGFBP5 *** <anterior>
['l

In [162]:
set3 = me[me.step==3].gene_symbol.unique()
set3 = list(set(set3) - set(set1))
set3 = list(set(set3) - set(set2))
set3_hits = {}
for gene in set3:
    hits = []
    for col in [x for x in studies.columns if '_P' not in x]:
        slist = [str(x).upper() for x in studies[col].values if pandas.notnull(x)]
        if gene.upper() in slist:
            hits.append(col)
    if len(hits)>0:
        ap = me[me.gene_symbol==gene]['ant-post'].values[0]
        print('***',gene,'***','<%s>'%ap)
        print(hits,'\n\n')
        set3_hits.update({gene: hits})

*** TOX *** <posterior>
['Cembrowski2013_ventral'] 


*** VAV3 *** <posterior>
['lee2017_dorsal'] 


*** CAMK2A *** <posterior>
['Leonardo_2006_dorsal_top1p', 'lee2017_dorsal'] 


*** LUM *** <posterior>
['Christensen_v', 'lee2017_ventral'] 


*** HOMER2 *** <posterior>
['Leonardo_2006_dorsal_top1p'] 


*** PPAPDC1A *** <anterior>
['lee2017_ventral'] 


*** KCNF1 *** <posterior>
['lee2017_ventral', 'Cembrowski2013_ventral'] 


*** BACE2 *** <anterior>
['lee2017_ventral'] 


*** EPHA5 *** <posterior>
['lee2017_dorsal'] 


*** ACVR1C *** <anterior>
['Cembrowski2013_ventral'] 


*** KCNJ16 *** <anterior>
['lee2017_ventral'] 


*** PID1 *** <posterior>
['Cembrowski2013_dorsal'] 


*** CAMK4 *** <posterior>
['lee2017_dorsal'] 


*** KL *** <anterior>
['Leonardo_2006', 'Leonardo_2006_dorsal', 'Leonardo_2006_ventral', 'Leonardo_2006_dorsal_top1p', 'Cembrowski2013_dorsal'] 


*** CRIM1 *** <anterior>
['lee2017_dorsal'] 


*** NTS *** <posterior>
['Oreilly_D', 'Cembrowski2017_D', 'lee2017_dorsa